In [ ]:
pip install opencv-python numpy tqdm albumentations pandas

In [6]:
!pip install "numpy<2.0"

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.1 which is incompatible.


In [1]:
import os
import cv2
import json
import glob
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm # Progress bar untuk notebook
from pathlib import Path

# --- KONFIGURASI PATH (SESUAIKAN DISINI) ---
# Folder dataset Virtual (Koryakinp)
RAW_VIRTUAL_TRAIN = r"D:\Projek uas Image Processing\Virtual_Chess\train" 
RAW_VIRTUAL_TEST  = r"D:\Projek uas Image Processing\Virtual_Chess\test"

# Folder dataset Synthetic (Thefamousrat)
RAW_SYNTH_DATA = r"D:\Projek uas Image Processing\Synthetic_Chess\data"

# Folder Output Final
OUTPUT_DIR = "yolo_dataset"

# Mapping Kelas (Standar Universal)
CLASS_MAP = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,  # Putih
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11 # Hitam
}
# Mapping nama di JSON Synthetic ke kode FEN
SYNTH_NAME_MAP = {
    "white_pawn": 'P', "white_knight": 'N', "white_bishop": 'B', 
    "white_rook": 'R', "white_queen": 'Q', "white_king": 'K',
    "black_pawn": 'p', "black_knight": 'n', "black_bishop": 'b', 
    "black_rook": 'r', "black_queen": 'q', "black_king": 'k'
}

# Buat struktur folder output
for split in ['train', 'val']:
    os.makedirs(f"{OUTPUT_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/labels/{split}", exist_ok=True)

print("✅ Setup direktori selesai.")

✅ Setup direktori selesai.


In [2]:
def fen_to_bboxes_flat(fen, img_w=1280, img_h=1280):
    """
    Untuk Dataset Virtual (Top-Down / Flat).
    Mengubah string FEN menjadi list YOLO bboxes berdasarkan Grid 8x8.
    """
    rows = fen.split('-') # Dataset ini pakai '-' sebagai pemisah, bukan '/'
    if len(rows) != 8: return [] # Validasi error

    bboxes = []
    square_w = img_w / 8
    square_h = img_h / 8
    
    for r, row_str in enumerate(rows): # r: 0-7
        c = 0 
        for char in row_str:
            if char.isdigit():
                c += int(char)
            else:
                if char in CLASS_MAP:
                    class_id = CLASS_MAP[char]
                    # Hitung center & size (Normalized 0-1)
                    x_center = (c * square_w + square_w / 2) / img_w
                    y_center = (r * square_h + square_h / 2) / img_h
                    w = square_w / img_w
                    h = square_h / img_h
                    bboxes.append([class_id, x_center, y_center, w, h])
                c += 1
    return bboxes

def calculate_homography_box(corners_norm, cell_x, cell_y, img_w, img_h):
    """
    Untuk Dataset Synthetic (Perspektif).
    Menghitung posisi kotak bidak berdasarkan 4 sudut papan menggunakan Homography.
    """
    # 1. Definisi Sudut Papan Ideal (Koordinat Papan Catur 8x8)
    # Urutan: Top-Left, Top-Right, Bottom-Right, Bottom-Left
    src_pts = np.array([[0, 0], [8, 0], [8, 8], [0, 8]], dtype=np.float32)
    
    # 2. Sudut Papan di Gambar (dari JSON) -> Denormalisasi ke Pixel
    dst_pts = np.array(corners_norm, dtype=np.float32)
    dst_pts[:, 0] *= img_w
    dst_pts[:, 1] *= img_h
    
    # 3. Hitung Matriks Homography
    M, _ = cv2.findHomography(src_pts, dst_pts)
    
    # 4. Tentukan koordinat sel target (misal E4 -> x=4, y=4)
    # Kita ambil 4 titik sudut sel tersebut
    cell_corners = np.array([
        [cell_x, cell_y],         # TL
        [cell_x + 1, cell_y],     # TR
        [cell_x + 1, cell_y + 1], # BR
        [cell_x, cell_y + 1]      # BL
    ], dtype=np.float32).reshape(-1, 1, 2)
    
    # 5. Proyeksikan (Transform) titik sel ke gambar perspektif
    transformed_corners = cv2.perspectiveTransform(cell_corners, M)
    
    # 6. Buat Bounding Box yang melingkupi area yang terproyeksi
    pts = transformed_corners.reshape(-1, 2)
    x_min, y_min = np.min(pts, axis=0)
    x_max, y_max = np.max(pts, axis=0)
    
    # Konversi ke YOLO format (normalized)
    box_w = (x_max - x_min)
    box_h = (y_max - y_min)
    x_c = x_min + box_w / 2
    y_c = y_min + box_h / 2
    
    return x_c/img_w, y_c/img_h, box_w/img_w, box_h/img_h

In [3]:
def fen_to_bboxes_flat(fen, img_w=1280, img_h=1280):
    """
    Untuk Dataset Virtual (Top-Down / Flat).
    Mengubah string FEN menjadi list YOLO bboxes berdasarkan Grid 8x8.
    """
    rows = fen.split('-') # Dataset ini pakai '-' sebagai pemisah, bukan '/'
    if len(rows) != 8: return [] # Validasi error

    bboxes = []
    square_w = img_w / 8
    square_h = img_h / 8
    
    for r, row_str in enumerate(rows): # r: 0-7
        c = 0 
        for char in row_str:
            if char.isdigit():
                c += int(char)
            else:
                if char in CLASS_MAP:
                    class_id = CLASS_MAP[char]
                    # Hitung center & size (Normalized 0-1)
                    x_center = (c * square_w + square_w / 2) / img_w
                    y_center = (r * square_h + square_h / 2) / img_h
                    w = square_w / img_w
                    h = square_h / img_h
                    bboxes.append([class_id, x_center, y_center, w, h])
                c += 1
    return bboxes

def calculate_homography_box(corners_norm, cell_x, cell_y, img_w, img_h):
    """
    Untuk Dataset Synthetic (Perspektif).
    Menghitung posisi kotak bidak berdasarkan 4 sudut papan menggunakan Homography.
    """
    # 1. Definisi Sudut Papan Ideal (Koordinat Papan Catur 8x8)
    # Urutan: Top-Left, Top-Right, Bottom-Right, Bottom-Left
    src_pts = np.array([[0, 0], [8, 0], [8, 8], [0, 8]], dtype=np.float32)
    
    # 2. Sudut Papan di Gambar (dari JSON) -> Denormalisasi ke Pixel
    dst_pts = np.array(corners_norm, dtype=np.float32)
    dst_pts[:, 0] *= img_w
    dst_pts[:, 1] *= img_h
    
    # 3. Hitung Matriks Homography
    M, _ = cv2.findHomography(src_pts, dst_pts)
    
    # 4. Tentukan koordinat sel target (misal E4 -> x=4, y=4)
    # Kita ambil 4 titik sudut sel tersebut
    cell_corners = np.array([
        [cell_x, cell_y],         # TL
        [cell_x + 1, cell_y],     # TR
        [cell_x + 1, cell_y + 1], # BR
        [cell_x, cell_y + 1]      # BL
    ], dtype=np.float32).reshape(-1, 1, 2)
    
    # 5. Proyeksikan (Transform) titik sel ke gambar perspektif
    transformed_corners = cv2.perspectiveTransform(cell_corners, M)
    
    # 6. Buat Bounding Box yang melingkupi area yang terproyeksi
    pts = transformed_corners.reshape(-1, 2)
    x_min, y_min = np.min(pts, axis=0)
    x_max, y_max = np.max(pts, axis=0)
    
    # Konversi ke YOLO format (normalized)
    box_w = (x_max - x_min)
    box_h = (y_max - y_min)
    x_c = x_min + box_w / 2
    y_c = y_min + box_h / 2
    
    return x_c/img_w, y_c/img_h, box_w/img_w, box_h/img_h

In [4]:
def process_virtual_folder(source_folder, split_name):
    print(f"🔄 Processing Virtual Dataset: {split_name}...")
    
    # Ambil semua gambar jpeg
    images = glob.glob(os.path.join(source_folder, "*.jpeg")) + glob.glob(os.path.join(source_folder, "*.jpg"))
    
    for img_path in tqdm(images):
        filename = os.path.basename(img_path)
        
        # 1. Parsing FEN dari nama file
        # Contoh: rnbqkbnr-pppppppp-8-8-8-8-PPPPPPPP-RNBQKBNR.jpeg
        fen_clean = os.path.splitext(filename)[0]
        
        # 2. Load Gambar untuk cek ukuran (biasanya fixed, tapi aman dicek)
        img = cv2.imread(img_path)
        if img is None: continue
        h, w, _ = img.shape
        
        # 3. Generate Labels
        bboxes = fen_to_bboxes_flat(fen_clean, w, h)
        
        # 4. Simpan (Copy gambar & Write .txt)
        # Kita beri prefix 'virt_' biar tidak bentrok nama
        new_filename = f"virt_{filename}"
        shutil.copy(img_path, f"{OUTPUT_DIR}/images/{split_name}/{new_filename}")
        
        txt_name = os.path.splitext(new_filename)[0] + ".txt"
        with open(f"{OUTPUT_DIR}/labels/{split_name}/{txt_name}", 'w') as f:
            for b in bboxes:
                f.write(f"{b[0]} {b[1]:.6f} {b[2]:.6f} {b[3]:.6f} {b[4]:.6f}\n")

# Eksekusi
process_virtual_folder(RAW_VIRTUAL_TRAIN, 'train') # Masuk ke folder train
process_virtual_folder(RAW_VIRTUAL_TEST, 'val')    # Masuk ke folder val

🔄 Processing Virtual Dataset: train...


  0%|          | 0/80000 [00:00<?, ?it/s]

🔄 Processing Virtual Dataset: val...


  0%|          | 0/20000 [00:00<?, ?it/s]

In [5]:
def process_synthetic_data(source_folder, split_ratio=0.9):
    print("🔄 Processing Synthetic Dataset...")
    
    # Cari pasangan JSON dan JPG
    json_files = glob.glob(os.path.join(source_folder, "*.json"))
    
    # Mapping Huruf Kolom ke Index (A=0, B=1, dst)
    col_map = {chr(65+i): i for i in range(8)} # {'A':0, 'B':1, ...}
    
    count_train = 0
    
    for json_path in tqdm(json_files):
        # Cari gambar pasangannya
        base_name = os.path.splitext(os.path.basename(json_path))[0]
        img_path = os.path.join(source_folder, base_name + ".jpeg")
        if not os.path.exists(img_path):
            img_path = os.path.join(source_folder, base_name + ".jpg")
            if not os.path.exists(img_path): continue
        
        # Load JSON & Image
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        img = cv2.imread(img_path)
        if img is None: continue
        h, w, _ = img.shape
        
        # Data Extraction
        corners = data.get('corners') # [[x,y], [x,y], ...] normalized
        config = data.get('config')   # {"A1": "white_rook", ...}
        
        if not corners or not config: continue
        
        yolo_labels = []
        
        # Loop setiap bidak di config
        for cell_name, piece_name in config.items():
            # Parse cell name (e.g., "E4")
            col_char = cell_name[0] # "E"
            row_char = cell_name[1] # "4"
            
            # Konversi ke koordinat grid (0-7)
            # A1 di deskripsi dataset = [0,0]. Biasanya A=0 (x), 1=0 (y).
            grid_x = col_map[col_char] 
            grid_y = int(row_char) - 1 
            
            # Map nama bidak ke ID
            if piece_name not in SYNTH_NAME_MAP: continue
            fen_char = SYNTH_NAME_MAP[piece_name]
            class_id = CLASS_MAP[fen_char]
            
            # HITUNG GEOMETRI (Homography)
            # Input: corners dataset, posisi grid x,y
            xc, yc, bw, bh = calculate_homography_box(corners, grid_x, grid_y, w, h)
            
            # Validasi agar tidak keluar gambar
            xc = max(0, min(1, xc))
            yc = max(0, min(1, yc))
            bw = max(0, min(1, bw))
            bh = max(0, min(1, bh))
            
            yolo_labels.append(f"{class_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")
            
        # Simpan Data (Random Split)
        split = 'train' if np.random.rand() < split_ratio else 'val'
        
        new_filename = f"synth_{base_name}.jpg"
        cv2.imwrite(f"{OUTPUT_DIR}/images/{split}/{new_filename}", img)
        
        txt_name = f"synth_{base_name}.txt"
        with open(f"{OUTPUT_DIR}/labels/{split}/{txt_name}", 'w') as f:
            f.write('\n'.join(yolo_labels))

# Eksekusi
process_synthetic_data(RAW_SYNTH_DATA)

🔄 Processing Synthetic Dataset...


  0%|          | 0/1944 [00:00<?, ?it/s]

In [ ]:
from ultralytics import YOLO
import glob
from IPython.display import Image, display
import os

# 1. Load Model Pre-trained (Masih "bodoh" soal catur)
model = YOLO("yolov8n.pt") 

print("🚀 Memulai validasi sebelum training (Baseline)...")
print("⚠️ Harap maklum jika hasilnya 0% atau error, karena model belum kenal kelas catur.")

# 2. Jalankan Validasi
# fraction=0.1 artinya kita cuma pakai 10% data validasi biar cepat
# plots=True wajib agar YOLO men-generate gambar confusion matrix
results = model.val(data=f"{OUTPUT_DIR}/dataset.yaml", imgsz=640, fraction=0.1, plots=True)

print("✅ Validasi selesai.")

In [ ]:
# 3. Cari folder hasil validasi terakhir
# Biasanya ada di runs/detect/val, val2, val3...
val_folders = sorted(glob.glob('runs/detect/val*'), key=os.path.getmtime)

if val_folders:
    latest_run = val_folders[-1]
    cm_path = os.path.join(latest_run, 'confusion_matrix.png')
    
    if os.path.exists(cm_path):
        print(f"📊 Menampilkan Confusion Matrix dari: {cm_path}")
        display(Image(filename=cm_path, width=800))
    else:
        print("⚠️ Gambar confusion_matrix.png tidak ditemukan. Mungkin model tidak mendeteksi apapun (semua background).")
        # Coba tampilkan confusion matrix ternormalisasi jika ada
        cm_norm_path = os.path.join(latest_run, 'confusion_matrix_normalized.png')
        if os.path.exists(cm_norm_path):
            display(Image(filename=cm_norm_path, width=800))
else:
    print("❌ Belum ada folder runs/detect/ ditemukan.")

In [ ]:
# Ensure required objects exist and select proper device (CPU if no CUDA)
import torch
try:
    from ultralytics import YOLO
except Exception:
    # If ultralytics is not installed, user must install it in the environment.
    raise

# Ensure yaml_path exists (fallback to expected default)
yaml_path = globals().get('yaml_path', 'yolo_dataset/dataset.yaml')

# Ensure model exists; if not, load a small pretrained model
if 'model' not in globals() or model is None:
    model = YOLO('yolov8n.pt')  # download/load a small YOLOv8 model

# Pick device safely: use CPU when no CUDA devices are available
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    device = '0'  # first GPU (string form)
    print(f"Using CUDA device(s): {device}")
else:
    device = 'cpu'
    print("CUDA not available, using CPU")

results = model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,        # Sesuaikan dengan VRAM. Coba 16, 32, atau 64. Semakin besar semakin cepat (selama gak Out of Memory).
    workers=4,       # Jumlah core CPU untuk load data. (Di Windows, set 0 jika error, tapi 4 lebih cepat)
    cache=True,      # Cache gambar ke RAM (Sangat mempercepat jika RAM > 16GB)
    device=device,   # Use selected device
    patience=5       # Stop training jika akurasi tidak naik selama 5 epoch (Early Stopping)
)